In [27]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup

In [28]:
link = 'https://www.worldometers.info/coronavirus/'
page = requests.get(link)
page.status_code

200

In [29]:
soup = BeautifulSoup(page.content)

In [30]:
table = soup.find('table', attrs={'id': 'main_table_countries_yesterday'})

In [31]:
rows = table.find_all("tr", attrs={"style": ""})

In [32]:
data = []
for i,item in enumerate(rows):
    if i == 0:
        data.append(item.text.strip().split("\n")[1:17])
        data[0][12]+=data[0][13]
        data[0]=data[0][:13]+data[0][-1:]
        data[0].append("Area")
    else:
        data.append(item.text.strip().split("\n")[1:16])

In [33]:
df = pd.DataFrame(data)
df = pd.DataFrame(data[2:-1], columns=data[0][:15])
df = df.rename(columns={'Tot\xa0Cases/1M pop':'TotCases/1M pop'})
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Area
0,China,"110,258",+294,"4,636",,"102,318",+84,"3,304",18,77,3,"160,000,000","111,163","1,439,323,776",Asia
1,USA,"80,895,651","+45,698","983,486","+1,454","54,344,934","+207,970","25,567,231","6,693","242,027","2,942","955,780,795","2,859,549","334,241,797",North America
2,India,"42,957,477","+5,921","514,908",+288,"42,378,721","+11,651","63,848","8,944","30,626",367,"770,973,356","549,658","1,402,643,172",Asia
3,Brazil,"28,974,315","+68,101","651,343",+697,"26,909,481","+99,195","1,413,491","8,318","134,714","3,028","63,776,166","296,522","215,080,841",South America
4,France,"22,958,320","+57,789","139,123",+181,"21,483,323","+118,431","1,335,874","2,484","350,431","2,124","246,629,975","3,764,506","65,514,564",Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,Macao,82,,,,79,,3,,123,,"5,318","8,008","664,102",Asia
217,Samoa,33,,,,28,,5,,164,,,,"200,614",Australia/Oceania
218,Vatican City,29,,,,28,,1,,"36,070",,,,804,Europe
219,Vanuatu,18,,1,,6,,11,,56,3,"23,000","72,075","319,112",Australia/Oceania


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Country,Other     221 non-null    object
 1   TotalCases        221 non-null    object
 2   NewCases          221 non-null    object
 3   TotalDeaths       221 non-null    object
 4   NewDeaths         221 non-null    object
 5   TotalRecovered    221 non-null    object
 6   NewRecovered      221 non-null    object
 7   ActiveCases       221 non-null    object
 8   Serious,Critical  221 non-null    object
 9   TotCases/1M pop   221 non-null    object
 10  Deaths/1M pop     221 non-null    object
 11  TotalTests        221 non-null    object
 12  Tests/1M pop      221 non-null    object
 13  Population        221 non-null    object
 14  Area              221 non-null    object
dtypes: object(15)
memory usage: 26.0+ KB


In [35]:
def replaceString(s:str):
    s = s.replace('+', '')
    s = s.replace(',', '')
    s = s.strip()
    return s

for col in df.columns:
    df[col] = df[col].apply(lambda s: replaceString(s))

In [36]:
df = df.replace(['', 'N/A'], np.nan)
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Area
0,China,110258,294,4636,NaN,102318,84,3304,18,77,3,160000000,111163,1439323776,Asia
1,USA,80895651,45698,983486,1454,54344934,207970,25567231,6693,242027,2942,955780795,2859549,334241797,North America
2,India,42957477,5921,514908,288,42378721,11651,63848,8944,30626,367,770973356,549658,1402643172,Asia
3,Brazil,28974315,68101,651343,697,26909481,99195,1413491,8318,134714,3028,63776166,296522,215080841,South America
4,France,22958320,57789,139123,181,21483323,118431,1335874,2484,350431,2124,246629975,3764506,65514564,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,Macao,82,NaN,NaN,NaN,79,NaN,3,NaN,123,NaN,5318,8008,664102,Asia
217,Samoa,33,NaN,NaN,NaN,28,NaN,5,NaN,164,NaN,NaN,NaN,200614,Australia/Oceania
218,Vatican City,29,NaN,NaN,NaN,28,NaN,1,NaN,36070,NaN,NaN,NaN,804,Europe
219,Vanuatu,18,NaN,1,NaN,6,NaN,11,NaN,56,3,23000,72075,319112,Australia/Oceania


In [37]:
columns = df.columns
for i in range(1, len(columns) - 1):
    df[columns[i]] = df[columns[i]].astype(float)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country,Other     221 non-null    object 
 1   TotalCases        221 non-null    float64
 2   NewCases          159 non-null    float64
 3   TotalDeaths       215 non-null    float64
 4   NewDeaths         106 non-null    float64
 5   TotalRecovered    213 non-null    float64
 6   NewRecovered      136 non-null    float64
 7   ActiveCases       213 non-null    float64
 8   Serious,Critical  163 non-null    float64
 9   TotCases/1M pop   221 non-null    float64
 10  Deaths/1M pop     215 non-null    float64
 11  TotalTests        211 non-null    float64
 12  Tests/1M pop      211 non-null    float64
 13  Population        221 non-null    float64
 14  Area              221 non-null    object 
dtypes: float64(13), object(2)
memory usage: 26.0+ KB


In [39]:
df.to_csv(r'./data.csv',index=False)